In [12]:
import math
import os.path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf
# calculate accuracy
from sklearn import metrics

In [25]:
f_predpath = open('./Predicate_paths/city_capitals_1.csv','r')
f_predpath = f_predpath.readlines()
f_predpath = f_predpath[1:]
f_predpath = [f_predpath[i].rstrip('\n').split(',')[1:] for i in range(len(f_predpath))]
for i in range(len(f_predpath)):
    for j in range(len(f_predpath[0])):
        if (f_predpath[i][j] == 'TRUE'): 
            f_predpath[i][j] = 1
        if (f_predpath[i][j] == 'FALSE'):
            f_predpath[i][j] = 0
        f_predpath[i][j] = int(f_predpath[i][j])
train_predpath = np.array(f_predpath)


f_triple = open('./data_id/city_capital.csv','r')
f_triple  = f_triple.readlines()
f_triple = f_triple[1:]
f_triple = [f_triple[i].rstrip('\n').split(',')[:-1] for i in range(len(f_triple))]
for i in range(len(f_triple)):
    for j in range(len(f_triple[0])):
        f_triple[i][j] = int(f_triple[i][j])
f_triple = np.array(f_triple)
hrt_triples = f_triple

n_predicate = train_predpath.shape[1]-1
print("N_TRAIN_TRIPLES: %d" % train_predpath.shape[0])

N_TRAIN_TRIPLES: 259


In [115]:
# Parameters
learning_rate = 0.1
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 100 # 1st layer number of neurons
#n_hidden_2 = 1 # 2nd layer number of neurons
n_input =  n_predicate# MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

embed_dim = n_hidden_1
bound = 6 / math.sqrt(n_hidden_1)

In [116]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_uniform([n_input, n_hidden_1], minval=-bound, maxval=bound, seed=345)),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([n_hidden_1, n_classes], minval=-bound, maxval=bound, seed=345))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #layer_1 = tf.matmul(x, weights['h1'])
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, weights['out'])  + biases['out']
    #out_layer = tf.matmul(layer_1, weights['out']) 
    return out_layer

# Construct model
logits = multilayer_perceptron(X)
    
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

In [117]:
with tf.Session() as session:
    #tf.global_variables_initializer().run()
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("Initializing 10-folds training data...")
    i_fold = 1
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        print("Fold ", i_fold)  
        tf.global_variables_initializer().run()
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))
        for epoch in range(training_epochs):

            accu_loss = 0.
            ninst = 0                 
            #print("Minibatches training... iteration: ", n_iter)           
            head_unique = np.unique(train_tiples[:,0])
            for i_head in head_unique:
                X_batch = train_predicates[train_tiples[:,0]==i_head,1:]
                tmp = train_predicates[train_tiples[:,0]==i_head,0]
                Y_labels = np.zeros([X_batch.shape[0], n_classes])
                for j in range(X_batch.shape[0]):
                    Y_labels[j,tmp[j]] = 1.
                _, c = session.run([train_op, loss_op], feed_dict = 
                                   {X: X_batch, Y: Y_labels})
                #accu_loss += l
            #print("Loss ", accu_loss)
        i_fold = i_fold + 1
        
        pred = tf.nn.softmax(logits)  # Apply softmax to logits
        y_p = tf.argmax(pred, 1)
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tmp_ = test_predicates[:,0]
        test_labels = np.zeros([len(tmp_), n_classes])
        for j in range(len(tmp_)):
            test_labels[j,tmp_[j]] = 1.
        print("Accuracy:", accuracy.eval({X: test_predicates[:,1:], Y: test_labels}))
        y_pred = session.run([y_p], feed_dict={X: test_predicates[:,1:]})[0]
        pred_proba = session.run([pred], feed_dict={X: test_predicates[:,1:]})[0]
        y_true = np.argmax(test_labels,1)
        TP_i = np.count_nonzero(y_pred * y_true)
        TN_i = np.count_nonzero((y_pred - 1) * (y_true - 1))
        FP_i = np.count_nonzero(y_pred * (y_true - 1))
        FN_i = np.count_nonzero((y_pred - 1) * y_true)
        auc_score += metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1])
        print("AUROC: ", metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1]))
        TP+=TP_i
        TN+=TN_i
        FP+=FP_i
        FN+= FN_i
        print("TP: ", TP_i,", FN: ",FN_i,", TN: ", TN_i,", FP: ",FP_i)
    print("TP: ", TP,", FN: ",FN,", TN: ", TN,", FP: ",FP)
    print auc_score/10

Initializing 10-folds training data...
('Fold ', 1)
('Label 1: ', 5, ' -- Label 0: ', 21)
('Accuracy:', 0.88461536)
('AUROC: ', 0.94285714285714284)
('TP: ', 4, ', FN: ', 1, ', TN: ', 19, ', FP: ', 2)
('Fold ', 2)
('Label 1: ', 7, ' -- Label 0: ', 19)
('Accuracy:', 0.92307693)
('AUROC: ', 0.96240601503759393)
('TP: ', 6, ', FN: ', 1, ', TN: ', 18, ', FP: ', 1)
('Fold ', 3)
('Label 1: ', 6, ' -- Label 0: ', 20)
('Accuracy:', 0.88461536)
('AUROC: ', 0.75)
('TP: ', 3, ', FN: ', 3, ', TN: ', 20, ', FP: ', 0)
('Fold ', 4)
('Label 1: ', 5, ' -- Label 0: ', 21)
('Accuracy:', 0.92307693)
('AUROC: ', 0.95238095238095244)
('TP: ', 5, ', FN: ', 0, ', TN: ', 19, ', FP: ', 2)
('Fold ', 5)
('Label 1: ', 4, ' -- Label 0: ', 22)
('Accuracy:', 0.92307693)
('AUROC: ', 0.85227272727272729)
('TP: ', 3, ', FN: ', 1, ', TN: ', 21, ', FP: ', 1)
('Fold ', 6)
('Label 1: ', 7, ' -- Label 0: ', 19)
('Accuracy:', 0.84615386)
('AUROC: ', 0.75939849624060152)
('TP: ', 4, ', FN: ', 3, ', TN: ', 18, ', FP: ', 1)
('Fo

In [34]:
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("Initializing 10-folds training data...")
    i_fold = 1
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    Confusion_mat = np.zeros([2,2])
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        print("Fold ", i_fold)  
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))

        logistic = LogisticRegression()
        logistic.fit(train_predicates[:,1:], train_predicates[:,0])
        print logistic
        i_fold = i_fold + 1
        
        print(metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:,1]))
        auc_score += metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:, 1])
        Confusion_mat += metrics.confusion_matrix(test_predicates[:,0], logistic.predict(test_predicates[:,1:]))
    print Confusion_mat
    print auc_score/10
                                                    


Initializing 10-folds training data...
('Fold ', 1)
('Label 1: ', 5, ' -- Label 0: ', 21)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.780952380952
('Fold ', 2)
('Label 1: ', 7, ' -- Label 0: ', 19)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.842105263158
('Fold ', 3)
('Label 1: ', 6, ' -- Label 0: ', 20)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, w

In [35]:
logistic.predict_proba(test_predicates[:,1:])

array([[  9.97976879e-01,   2.02312105e-03],
       [  9.48479722e-01,   5.15202779e-02],
       [  4.31231173e-02,   9.56876883e-01],
       [  1.54102064e-10,   1.00000000e+00],
       [  9.84247248e-01,   1.57527519e-02],
       [  1.00000000e+00,   1.19303862e-10],
       [  9.71466387e-01,   2.85336127e-02],
       [  9.79940755e-01,   2.00592454e-02],
       [  1.00000000e+00,   2.34271958e-22],
       [  9.96812177e-01,   3.18782272e-03],
       [  1.32890370e-01,   8.67109630e-01],
       [  9.51230635e-01,   4.87693652e-02],
       [  9.96726576e-01,   3.27342442e-03],
       [  9.93972060e-01,   6.02794005e-03],
       [  2.77708392e-01,   7.22291608e-01],
       [  9.83800867e-01,   1.61991334e-02],
       [  9.99918235e-01,   8.17651777e-05],
       [  1.75674580e-01,   8.24325420e-01],
       [  9.98502970e-01,   1.49702988e-03],
       [  9.99912332e-01,   8.76677135e-05],
       [  9.89522454e-01,   1.04775457e-02],
       [  9.94863187e-01,   5.13681295e-03],
       [  

In [36]:
logistic.predict(test_predicates[:,1:])

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0])

In [177]:
f_predpath = open('./Predicate_paths/capitals_3.csv','r')
f_predpath = f_predpath.readlines()
f_predpath = f_predpath[1:]
f_predpath = [f_predpath[i].rstrip('\n').split(',')[1:] for i in range(len(f_predpath))]
for i in range(len(f_predpath)):
    for j in range(len(f_predpath[0])):
        if (f_predpath[i][j] == 'TRUE'): 
            f_predpath[i][j] = 1
        if (f_predpath[i][j] == 'FALSE'):
            f_predpath[i][j] = 0
        f_predpath[i][j] = int(f_predpath[i][j])
train_predpath = np.array(f_predpath)


f_triple = open('./data_id/largest_city_capitals.csv','r')
f_triple  = f_triple.readlines()
f_triple = f_triple[1:]
f_triple = [f_triple[i].rstrip('\n').split(',')[1:-1] for i in range(len(f_triple))]
for i in range(len(f_triple)):
    for j in range(len(f_triple[0])):
        f_triple[i][j] = int(f_triple[i][j])
f_triple = np.array(f_triple)
hrt_triples = f_triple

n_predicate = train_predpath.shape[1]-1
print("N_TRAIN_TRIPLES: %d" % train_predpath.shape[0])

N_TRAIN_TRIPLES: 266


In [122]:
train_predpath

array([[ 1, 20, 17, ...,  0,  0,  0],
       [ 1,  3,  8, ...,  0,  0,  0],
       [ 1, 55, 61, ...,  0,  0,  0],
       ..., 
       [ 0,  5,  6, ...,  0,  0,  0],
       [ 0,  2,  2, ...,  0,  0,  0],
       [ 0,  1,  4, ...,  0,  0,  0]])

In [123]:
kf = KFold(n_splits=10, shuffle = True, random_state=233)
print("Initializing 10-folds training data...")
i_fold = 1
TP = 0
TN = 0
FP = 0
FN = 0
Confusion_mat = np.zeros([2,2])
auc_score = 0.0
for i_train, i_test in kf.split(train_predpath):
    print("Fold ", i_fold)  
    train_predicates = np.array(train_predpath)[i_train]
    test_predicates = np.array(train_predpath)[i_test]
    train_tiples = np.array(hrt_triples)[i_train]    
    print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))

    logistic = LogisticRegression()
    logistic.fit(train_predicates[:,1:], train_predicates[:,0])
    i_fold = i_fold + 1

    print(metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:,1]))
    auc_score += metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:, 1])
    Confusion_mat += metrics.confusion_matrix(test_predicates[:,0], logistic.predict(test_predicates[:,1:]))
print Confusion_mat
print auc_score/10

Initializing 10-folds training data...
('Fold ', 1)
('Label 1: ', 5, ' -- Label 0: ', 22)
0.7
('Fold ', 2)
('Label 1: ', 7, ' -- Label 0: ', 20)
0.692857142857
('Fold ', 3)
('Label 1: ', 5, ' -- Label 0: ', 22)
1.0
('Fold ', 4)
('Label 1: ', 4, ' -- Label 0: ', 23)
0.945652173913
('Fold ', 5)
('Label 1: ', 7, ' -- Label 0: ', 20)
0.921428571429
('Fold ', 6)
('Label 1: ', 6, ' -- Label 0: ', 21)
0.801587301587
('Fold ', 7)
('Label 1: ', 4, ' -- Label 0: ', 22)
1.0
('Fold ', 8)
('Label 1: ', 3, ' -- Label 0: ', 23)
0.695652173913
('Fold ', 9)
('Label 1: ', 3, ' -- Label 0: ', 23)
0.95652173913
('Fold ', 10)
('Label 1: ', 6, ' -- Label 0: ', 20)
0.708333333333
[[ 205.   11.]
 [  18.   32.]]
0.842203243616


In [173]:
from sklearn.cross_validation import cross_val_score
logreg = LogisticRegression()
cross_val_score(logreg, np.array(train_predpath)[:,1:], np.array(train_predpath)[:,0], cv=10, scoring='roc_auc').mean()



0.88257575757575757

In [187]:
# Parameters
learning_rate = 0.01
training_epochs = 500
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 200 # 1st layer number of neurons
#n_hidden_2 = 100 # 2nd layer number of neurons
n_input =  n_predicate# MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

embed_dim = n_hidden_1
bound = 6 / math.sqrt(n_hidden_1)

In [188]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_uniform([n_input, n_hidden_1], minval=-bound, maxval=bound, seed=345)),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([n_hidden_1, n_classes], minval=-bound, maxval=bound, seed=345))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    #layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.matmul(x, weights['h1'])
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    #out_layer = tf.matmul(layer_1, weights['out'])  + biases['out']
    out_layer = tf.matmul(layer_1, weights['out']) 
    return out_layer

# Construct model
logits = multilayer_perceptron(X)
    
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as session:
    #tf.global_variables_initializer().run()
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("Initializing 10-folds training data...")
    i_fold = 1
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        print("Fold ", i_fold)  
        tf.global_variables_initializer().run()
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))
        for epoch in range(training_epochs):

            accu_loss = 0.
            ninst = 0                 
            #print("Minibatches training... iteration: ", n_iter)           
            head_unique = np.unique(train_tiples[:,0])
            for i_head in head_unique:
                X_batch = train_predicates[train_tiples[:,0]==i_head,1:]
                tmp = train_predicates[train_tiples[:,0]==i_head,0]
                Y_labels = np.zeros([X_batch.shape[0], n_classes])
                for j in range(X_batch.shape[0]):
                    Y_labels[j,tmp[j]] = 1.
                _, c = session.run([train_op, loss_op], feed_dict = 
                                   {X: X_batch, Y: Y_labels})
                #accu_loss += l
            #print("Loss ", accu_loss)
        i_fold = i_fold + 1
        
        pred = tf.nn.softmax(logits)  # Apply softmax to logits
        y_p = tf.argmax(pred, 1)
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tmp_ = test_predicates[:,0]
        test_labels = np.zeros([len(tmp_), n_classes])
        for j in range(len(tmp_)):
            test_labels[j,tmp_[j]] = 1.
        print("Accuracy:", accuracy.eval({X: test_predicates[:,1:], Y: test_labels}))
        y_pred = session.run([y_p], feed_dict={X: test_predicates[:,1:]})[0]
        pred_proba = session.run([pred], feed_dict={X: test_predicates[:,1:]})[0]
        y_true = np.argmax(test_labels,1)
        TP_i = np.count_nonzero(y_pred * y_true)
        TN_i = np.count_nonzero((y_pred - 1) * (y_true - 1))
        FP_i = np.count_nonzero(y_pred * (y_true - 1))
        FN_i = np.count_nonzero((y_pred - 1) * y_true)
        auc_score += metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1])
        print("AUROC: ", metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1]))
        TP+=TP_i
        TN+=TN_i
        FP+=FP_i
        FN+= FN_i
        print("TP: ", TP_i,", FN: ",FN_i,", TN: ", TN_i,", FP: ",FP_i)
    print("TP: ", TP,", FN: ",FN,", TN: ", TN,", FP: ",FP)
    print auc_score/10

Initializing 10-folds training data...
('Fold ', 1)
('Label 1: ', 5, ' -- Label 0: ', 22)
('Accuracy:', 0.8888889)
('AUROC: ', 0.84090909090909083)
('TP: ', 4, ', FN: ', 1, ', TN: ', 20, ', FP: ', 2)
('Fold ', 2)
('Label 1: ', 7, ' -- Label 0: ', 20)
('Accuracy:', 0.85185188)
('AUROC: ', 0.76428571428571435)
('TP: ', 4, ', FN: ', 3, ', TN: ', 19, ', FP: ', 1)
('Fold ', 3)
('Label 1: ', 5, ' -- Label 0: ', 22)
('Accuracy:', 0.8888889)
('AUROC: ', 0.92727272727272725)
('TP: ', 4, ', FN: ', 1, ', TN: ', 20, ', FP: ', 2)
('Fold ', 4)
('Label 1: ', 4, ' -- Label 0: ', 23)
('Accuracy:', 0.8888889)
('AUROC: ', 0.71739130434782616)
('TP: ', 2, ', FN: ', 2, ', TN: ', 22, ', FP: ', 1)
('Fold ', 5)
